# Gaussian elimination and LU decomposition

Say we want to compute the solution of
$$Ax = b$$
for the vector $x$. We learn how to do this by transforming it to the problem of solving
$$U x = y$$
where $U$ is an upper-triangular matrix obtained by performing Gaussian elimiantion on $A$ and $y$ is obtained by performing the same operations on $b$. We can then use back substitution to solve $Ux=y$ more easily than solving $Ax=b$ directly.

This approach is directly related to the LU decomposition of a matrix, where we wish to factor a matrix $A$ into a product of a lower triangular matrix $L$ and an upper triangular matrix $U$ to give $A = LU$. To understand how to compute the LU decomposition of a matrix, let us start by reminding ourselves of how to do Gaussian elimination.

## Gaussian elimination by hand

To start, consider the following 3x3 matrix
$$ A = \begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6 \\ 7 & 8 & 10 \end{bmatrix}$$

1. Use Gaussian elimination to transform this by hand to an upper triangular matrix $U$ (in row echelon form). Record each elementary row operation you perform along the way.

2. Apply the same sequence of row operations to the vector
$$b = \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix}$$
to obtain the transformed vector $y$.

3. Use back substitution to solve $U x = y$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [60]:
A = np.array(([1,2,3],[4,5,6],[7,8,10]))
B=np.zeros((3,3))
B[0] = A[0]
B[1] = (A[1] - 4*A[0])
B[2] = (A[2] - 7*A[0] - 2*A[1] + 8*A[0])

# Do we divide by -3 in line 2

B

array([[ 1.,  2.,  3.],
       [ 0., -3., -6.],
       [ 0.,  0.,  1.]])

## Gaussian elimination in Python

We will now transform the previous algorithm into Python code. First of all we define the matrix $A$ and the vector $b$.

In [24]:
# using loops
U = np.array(([1,2,3],[4,5,6],[7,8,10]))

y = np.array(([1],[2],[3]))

n = np.shape(U)[1]

L = np.identity(n)


for k in range(0,n):
    for i in range(k+1,n):
        L[i,k] = U[i,k]/U[k,k]
        
        U[i,:] = U[i,:] - L[i,k]*U[k,:]
        y[i] = y[i] - L[i,k]*y[k]
        
print(U)
print()
print(L)

[[ 1  2  3]
 [ 0 -3 -6]
 [ 0  0  1]]

[[1. 0. 0.]
 [4. 1. 0.]
 [7. 2. 1.]]


In [25]:
n = len(U)
L = np.identity(n)
for k in range(0,n):
    for i in range(k+1,n):
        L[i,k] = U[i,k]/U[k,k]
        
        U[i,:] = U[i,:] - L[i,k]*U[k,:]
        y[i] = y[i] - L[i,k]*y[k]

In [33]:
print(L@U)

[[ 1.  2.  3.]
 [ 0. -3. -6.]
 [ 0.  0.  1.]]


Now perform Gaussian elimination and store the result in a matrix $U$ and a vector $y$. We keep track of the multiplication factors for each step in a matrix $L$.

If we consider how many operations this took, there are: ($n$ iterations of the outer loop) x ($n-(k+1)$ iterations of the inner loop) x (n multiplications for the subtraction). This means we require $\mathcal{O}(n^3)$ operations for the Gaussian elimination step.

Let us now solve for $x$ using back substitution on $U x = y$.

In [47]:
x = np.zeros((np.shape(U)[1],1))

x[n-1] = y[n-1] / U[n-1,n-1]

for i in range(n-2,-1,-1):
    x[i] = (y[i] - U[i,i+1:n]@x[i+1:n])/U[i,i]

print(x)

[[-0.33333333]
 [ 0.66666667]
 [ 0.        ]]


In [64]:
print(U@x)
print()
print(y)

[[ 1.]
 [-2.]
 [ 0.]]

[[ 1]
 [-2]
 [ 0]]


We can check that our original matrix is given by $A=LU$:

## Gaussian elimination by matrix multiplication

We could consider each of the steps in Gaussian elimination in terms of multiplication on the left by a sequence of *elementary elimination matrices*. These come in three forms:

1. Multiplying row $i$ by a scalar $c$: $\mathbf{r}_i \to c\, \mathbf{r}_i$. This is equivalent to pre-multiplying by a matrix with $1$'s along the diagonal and c in the $i$-th diagonal,$$E_1(i, c) = \begin{bmatrix}
  1 &        &   &   &   &        &   \\
    & \ddots &   &   &   &        &   \\
    &        & 1 &   &   &        &   \\
    &        &   & c &   &        &   \\
    &        &   &   & 1 &        &   \\
    &        &   &   &   & \ddots &   \\
    &        &   &   &   &        & 1
\end{bmatrix}$$
Note that the inverse is given by $E_1(c)^{-1} = E_1(c^{-1})$.

2. Add a multiple $c$ of row $j$ to row $i$: $\mathbf{r}_i \to \mathbf{r}_i  + c\, \mathbf{r}_j$. This is equivalent to premultiplying by a matrix with $1$'s along the diagonal and $c$ in $(i, j)$-th entry:
$$E_2(i,j,c) = \begin{bmatrix}
  1 &        &   &        &   &        &   \\
    & \ddots &   &        &   &        &   \\
    &        & 1 &        &   &        &   \\
    &        &   & \ddots &   &        &   \\
    &        & c &        & 1 &        &   \\
    &        &   &        &   & \ddots &   \\
    &        &   &        &   &        & 1
\end{bmatrix}$$
In this case the inverse is given by $E_2(c)^{-1} = E_2(-c)$.

3. Interchanging rows $i$ and $j$: $\mathbf{r}_i \leftrightarrow \mathbf{r}_j$. This is equivalent to pre-multiplying by a matrix which is the identity with rows $i$ and $j$ swapped: $$E_3(i,j) = \begin{bmatrix}
  1 &        &   &        &   &        &   \\
    & \ddots &   &        &   &        &   \\
    &        & 0 &        & 1 &        &   \\
    &        &   & \ddots &   &        &   \\
    &        & 1 &        & 0 &        &   \\
    &        &   &        &   & \ddots &   \\
    &        &   &        &   &        & 1
\end{bmatrix}$$
In this case the $E_3$ is a permutation matrix and it is its own inverse $E_3^{-1} = E_3$.

In [98]:
A1 = np.array(([1,2,3,5],[4,5,6,11],[7,8,10,13],[6,32,5,2]))
print(A1)

[[ 1  2  3  5]
 [ 4  5  6 11]
 [ 7  8 10 13]
 [ 6 32  5  2]]


In [99]:
z = np.identity(4)
z[1,1] = 4
print(z@A1)

# multiply row of A1 by a constant

[[ 1.  2.  3.  5.]
 [16. 20. 24. 44.]
 [ 7.  8. 10. 13.]
 [ 6. 32.  5.  2.]]


In [100]:
z = np.identity(4)
z[2,0] = 3
z@A1

# z[i,j] add a multiple of row j of A to row i of A

array([[ 1.,  2.,  3.,  5.],
       [ 4.,  5.,  6., 11.],
       [10., 14., 19., 28.],
       [ 6., 32.,  5.,  2.]])

In [109]:
z = np.zeros((4,4))
z[0,0]=1

z[2,2]=1
z[3,1] = 1
z[1,3] = 1
print(z)

# in z, only one 1 per row

[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]


Let's work out the sequence of elimination matrices we need to perform the Gaussian elimination from the previous example. First, we define Python functions produce each of the three types of elimination matrix:

Now, we can see that the Gaussian elimination steps correspond to
$$ U = E_2(2,1,-2) E_2(2,0,-7) E_2(1,0,-4) A$$

We therefore have
$$
\begin{aligned}
A &= [E_2(2,1,-2) E_2(2,0,-7) E_2(1,0,-4)]^{-1} U \\
  &= E_2(1,0,-4)^{-1} E_2(2,0,-7)^{-1} E_2(2,1,-2)^{-1} U \\
  &= E_2(1,0,4) E_2(2,0,7) E_2(2,1,2) U \\
  &= L U
\end{aligned}
$$
so we have $L$ in terms of elementry elimination matrices.

## LU decomposition and rank-1 matrices

In the lecture videos we emphasized the idea of matrix multiplication in terms of columns-times-rows and the related idea of breaking a matrix into a sum of rank-1 matrices. Now, let's see how this gives a different way of looking at the LU decomposition.

The idea is that we would like to split $A$ into a rank-1 piece that picks out the first row and first column, plus a rank-1 piece that picks out the next row and column, and so on:
$$
\begin{aligned}
A = \begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6 \\ 7 & 8 & 10 \end{bmatrix}
 &= \begin{bmatrix} 1 & 2 & 3 \\ 4 & \_ & \_ \\ 7 & \_ & \_ \end{bmatrix}
   + \begin{bmatrix} 0 & 0 & 0 \\ 0 & \_ & \_ \\ 0 & \_ & \_ \end{bmatrix} 
   + \begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & \_ \end{bmatrix}
\end{aligned}
$$
We can fill in all the blanks here by insisting that each term is rank-1 and that we recover $A$.

In [87]:
M1 = np.array([[1,2,3],[4,8,12],[7,14,21]])
M2 = np.array([[0,0,0],[0,-3,-6],[0,-6,-12]])
M3 = np.array([[0,0,0],[0,0,0],[0,0,1]])


print(M1)
print()
print(M2)
print()
print(M3)

[[ 1  2  3]
 [ 4  8 12]
 [ 7 14 21]]

[[  0   0   0]
 [  0  -3  -6]
 [  0  -6 -12]]

[[0 0 0]
 [0 0 0]
 [0 0 1]]


In [86]:
print(M1+M2+M3)

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8 10]]
